In [3]:
import pandas as pd

In [5]:
data = pd.read_csv("../data/processed/data_processed.csv")
data.sample(5)

,context_3,context_2,context_1,response
3357,NaN,NaN,NaN,"я про них и подумал, но ведь и не милфа так может"
11637,NaN,NaN,NaN,"Всем привет, тут резюмешка, пытаюсь пробиться ..."
1909,NaN,"Пробую тоже xgboost, таргет close, если прогно...",и как результаты?,"Ну за 2023 плюс, но исторический бэктест минус..."
2667,NaN,NaN,"Всем, привет! какой mAP(IoU=0.5) хорош для прода?",Ты про детекцию или инстанс сегментацию? На ск...
4754,NaN,NaN,NaN,"Подскажите, насколько хороша идея применять Mi..."


In [6]:
data.shape

(13832, 4)

In [9]:
data.head(60)

,context_3,context_2,context_1,response
0,NaN,NaN,NaN,"Флудилка, можно политику, срачи, ограничений нет"
1,NaN,NaN,"Флудилка, можно политику, срачи, ограничений нет",Мне кажется стоит отключить создание топиков д...
2,NaN,NaN,NaN,Ахахаахахах тупо всё почистил\nЕбать я злой
3,NaN,NaN,Ахахаахахах тупо всё почистил\nЕбать я злой,(ограничений нет)
4,NaN,NaN,NaN,TTS/ Stt/ voice движуха
5,NaN,NaN,TTS/ Stt/ voice движуха,А не стоит ли сделать отдельный топик по музык...
6,NaN,TTS/ Stt/ voice движуха,А не стоит ли сделать отдельный топик по музык...,ну там многое близко
7,TTS/ Stt/ voice движуха,А не стоит ли сделать отдельный топик по музык...,ну там многое близко,Но при этом есть отдельная конференция ISMIR и...
8,А не стоит ли сделать отдельный топик по музык...,ну там многое близко,Но при этом есть отдельная конференция ISMIR и...,Справедливо
9,NaN,NaN,NaN,политику можно? поехали


In [8]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

In [159]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [160]:
model_path = "../models/lora-5ep"

config = PeftConfig.from_pretrained(model_path)

In [161]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [162]:
model = PeftModel.from_pretrained(model, model_path)

In [163]:
input_str = "привет"

In [164]:
input_ids = tokenizer(input_str, return_tensors="pt")
generated_token_ids = model.generate(
            **input_ids,
            top_k=10,
            top_p=0.95,
            num_beams=3,
            num_return_sequences=1,
            do_sample=True,
            no_repeat_ngram_size=2,
            temperature=1.5,
            repetition_penalty=1.2,
            length_penalty=1.0,
            eos_token_id=50257,
            max_new_tokens=40,
        )
generated_output = tokenizer.decode(generated_token_ids[0], skip_special_tokens=True)
cutted_answer = generated_output
# cut off the answer if it contains the special tokens
# if "@@ПЕРВЫЙ@@" in cutted_answer:
#     cutted_answer = cutted_answer.split("@@ПЕРВЫЙ@@")[0]
# if "@@ВТОРОЙ@@" in cutted_answer:
#     cutted_answer = cutted_answer.split("@@ВТОРОЙ@@")[0]

cutted_answer

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


'привет@@ВТОРОЙ@@Привет, как дела? 🥺@@ПЕРВЫЙ@@'

In [165]:
input_str = "где веса ламы найти?"

chat_history_ids = "@@ПЕРВЫЙ@@ " + input_str + "@@ВТОРОЙ@@"
new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
generated_token_ids = model.generate(
        **new_input_ids,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=1,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.4,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=40,
        pad_token_id=tokenizer.eos_token_id
    )

context_with_response = [tokenizer.decode(generated_token_ids[i]) for i in range(len(generated_token_ids))]
context_with_response

['@@ПЕРВЫЙ@@ где веса ламы найти?@@ВТОРОЙ@@В гугле по картинкам.@@ПЕРВЫЙ@@']

In [124]:
import torch

torch.__version__

'2.0.1'

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-small')
inputs = tokenizer('@@ПЕРВЫЙ@@ где найти веса ламы? @@ВТОРОЙ@@', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=5,
    top_p=0.9,
    num_beams=3,
    num_return_sequences=1,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.7,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40,
    pad_token_id=tokenizer.eos_token_id
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['@@ПЕРВЫЙ@@ где найти веса ламы? @@ВТОРОЙ@@В интернете. Там много чего можно найти. Например, в гугле. Или в яндекс картинках.@@ПЕРВЫЙ@@']

In [29]:
import requests
requests.__version__

'2.28.2'

In [27]:
!pip3 install telebot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Using cached telebot-0.0.5-py3-none-any.whl (4.8 kB)
  Using cached pyTelegramBotAPI-4.13.0.tar.gz (232 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.13.0-py3-none-any.whl size=214546 sha256=42b335ca9dbe06e3b57c425bcf67dd685b28c9c9f6624720ac94fff15edef1c7
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/6d/05/82/5f51ca10e3939eb47f61064a76b1fb027ff69927a25a3c5190
Successfully built pyTelegramBotAPI

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLi